In [1]:
import pandas as pd

In [5]:
df = pd.read_csv("../../dataset/nou_hearings.csv")

df2 = df["actor"].unique()
pd.DataFrame(df2)

,0
0,Abelia
1,Advokatforeningen
2,Agder fylkeskommune
3,Akademikerne
4,Akershus fylkeskommune
...,...
77,Kvinnherad INP
78,Kviteseid Kommune
79,Landssamanslutninga av Vasskraftkommunar (LVK)...
80,Lars G Solvik
